# Initialize system

In [ ]:
### Setup the library ###
import pyrogue as pr
import os, sys
import matplotlib.pyplot as plt
import time
import datetime
import numpy as np
import math
import pprint
import inspect 
top_level=f'{os.getcwd()}/../'
rootTopLevel = top_level+'script/'
pr.addLibraryPath( rootTopLevel )

import setupLibPaths

import ePix320kM as devBoard

args = None

In [2]:
# ONLY RUN ONCE!
# Defining root
root =  devBoard.Root(   
        top_level  = rootTopLevel,
        dev        = '/dev/datadev_1',
        pollEn     = False,
        initRead   = True,
        serverPort = 9099)
root.start()

# example showing a read
AxiVersion = root.Core.AxiVersion
print ( '###################################################')
print ( '#             Firmware Version                    #')
print ( '###################################################')
AxiVersion.printStatus()
print ( '###################################################')

{'name': 'ScopeData0'}
{'name': 'ScopeData1'}
{'name': 'ScopeData2'}
{'name': 'ScopeData3'}
Root.Core.Si5345Pll.Page0.DataBlock.setPollInterval(1)
Start: Started zmqServer on ports 9099-9101
    To start a gui: python -m pyrogue gui --server='localhost:9099'
    To use a virtual client: client = pyrogue.interfaces.VirtualClient(addr='localhost', port=9099)
Root.Core.AxiVersion count reset called
###################################################
#             Firmware Version                    #
###################################################
Path         = Root.Core.AxiVersion
FwVersion    = 0x1000300
UpTime       = 1 day, 0:27:21
GitHash      = 0x9550c2a95f7e334393e52bfde914e26fb20f11b8
XilinxDnaId  = 0x400200000139d7204c81a145
FwTarget     = ePixHRM320k
BuildEnv     = Vivado v2023.1
BuildServer  = rdsrv314 (Ubuntu 20.04.6 LTS)
BuildDate    = Wed 17 Jan 2024 11:42:17 PM PST
Builder      = dnajjar
###################################################


In [3]:
# Useful short names
APP  = root.App
AXIV = root.Core.AxiVersion
ASICTOP = APP.AsicTop
TRIG = ASICTOP.TriggerRegisters
ASIC0 = APP.Mv2Asic[0]
ASIC1 = APP.Mv2Asic[1]
ASIC2 = APP.Mv2Asic[2]
ASIC3 = APP.Mv2Asic[3]
HSDAC = APP.Dac.FastDac 
PKREG0 = ASICTOP.DigAsicStrmRegisters0
PKREG1 = ASICTOP.DigAsicStrmRegisters1
PKREG2 = ASICTOP.DigAsicStrmRegisters2
PKREG3 = ASICTOP.DigAsicStrmRegisters3
BATCHER0 = ASICTOP.BatcherEventBuilder0
BATCHER1 = ASICTOP.BatcherEventBuilder1
BATCHER2 = ASICTOP.BatcherEventBuilder2
BATCHER3 = ASICTOP.BatcherEventBuilder3
DEBUG0 = root._dbg[0]
DEBUG1 = root._dbg[1]
DEBUG2 = root._dbg[2]
DEBUG3 = root._dbg[3]
DATARCV0 = root.DataReceiver0
DATARCV1 = root.DataReceiver1
DATARCV2 = root.DataReceiver2
DATARCV3 = root.DataReceiver3
FULLRATERCV0 = root.fullRateDataReceiver[0]
FULLRATERCV1 = root.fullRateDataReceiver[1]
FULLRATERCV2 = root.fullRateDataReceiver[2]
FULLRATERCV3 = root.fullRateDataReceiver[3]
DAC = APP.Dac
REGCTRL = ASICTOP.RegisterControlDualClock

In [4]:
# Configure clock to 168 MHz and configures all ASICS
root.InitASIC([4,1,1,1,1])

Rysync ASIC started
Init ASIC script started
Loading PLL configuration
Loading supply configuration
Loading /u1/lorenzor/epix-hr-m-320k/software/notebook/../script/../config/ePixHRM320k_PowerSupply_Enable.yml
Loading lane delay configurations
Loading /u1/lorenzor/epix-hr-m-320k/software/notebook/../script/../config/ePixHRM320k_SspMonGrp_carrier3.yml
Loading waveforms configuration
Loading /u1/lorenzor/epix-hr-m-320k/software/notebook/../script/../config/ePixHRM320k_RegisterControl.yml
Loading packet register configurations
Loading /u1/lorenzor/epix-hr-m-320k/software/notebook/../script/../config/ePixHRM320k_PacketRegisters.yml
Loading batcher configurations
Loading /u1/lorenzor/epix-hr-m-320k/software/notebook/../script/../config/ePixHRM320k_BatcherEventBuilder.yml
Taking asic off of reset
Loading ASICs and timing configuration
Loading /u1/lorenzor/epix-hr-m-320k/software/notebook/../script/../config/ePixHRM320k_ASIC_u1_PLLBypass.yml
Loading /u1/lorenzor/epix-hr-m-320k/software/noteboo

In [5]:
root.disableAndCleanAllFullRateDataRcv()
root.enableDataRcv(False)
root.enableDataDebug(False)

In [6]:
#run some triggers and exercise lanes and locks
frames = 100
rate = 5000

root.hwTrigger(frames, rate)

Stop Triggers command executed
Set Auto Trigger command executed
Start Auto Trigger command executed
Triggers sent: 100
Stop Triggers command executed


In [7]:
#get locked lanes
time.sleep(3)
root.getLaneLocks()

ASIC0: 0xffffff
ASIC1: 0xfffffa
ASIC2: 0xb6ffff
ASIC3: 0xbfffff


In [8]:
#Disabled lanes in ASICS
for asicIndex in range(4) : 
    root.App.AsicTop.DigAsicStrmRegisters0.enable.set(True)
    print("ASIC{}: {:#x}".format(asicIndex, getattr(root.App.AsicTop, f"DigAsicStrmRegisters{asicIndex}").DisableLane.get()))

ASIC0: 0x20000
ASIC1: 0x10045
ASIC2: 0x490000
ASIC3: 0x540140


In [9]:
#Enable data receivers and run some triggers
root.enableDataRcv(True)
root.enableAllAsics(True)
for i in range (5):
    root.Trigger()

descramble error
rawData length 6096
descramble error
rawData length 6096
descramble error
rawData length 6096


In [10]:
# Obtain descrambled single frame data from ASICs from DataReceiver. Data receiver is down sampled.
root.printDataReceiverStatus()
    
frame = [None for i in range(4)]
for asicIndex in range(4):
    frame[asicIndex] = getattr(root, f"DataReceiver{asicIndex}").Data.get()

Checkpoint: DataReceiver 0 has 1 frames
Checkpoint: DataReceiver 1 has 1 frames
Checkpoint: DataReceiver 2 has 1 frames
Checkpoint: DataReceiver 3 has 1 frames


# Here we select which ASIC we want to test

In [15]:
# Change these 3 lines when you want to select another ASIC
AsicNum = 1
dataDebug = DEBUG2
ASIC = ASIC2

# From here it's all the same
root.enableAllAsics(False)
root.enableAsic(AsicNum, True)
root.enableDataRcv(False)

dataDebug.enableDataDebug(True) ##### -- This was changed !!!
dataDebug.cleanData() ##### -- This was changed !!!

# Lorenzo - Helper functions

In [16]:
%run dataProcFunctions.ipynb     #this is were my function was stored

# Set registers and variables

In [17]:
setBestSettings(REGCTRL,ASIC0,ASIC1,ASIC2,ASIC3)

R0 Width set to 7.808uS
R0 Delay set to 1.280uS
Acq1 Width set to 3.584uS
Acq1 Delay set to 4.864uS
-------------------------
Baseline time:    3.5839999999999996 uS
Integration time: 3.584 uS
Sampling done at: 8.448 uS
-------------------------
ASIC configured


In [19]:
# root.enableDataRcv(False)
# root.disableAndCleanAllFullRateDataRcv()
# root.enableAllAsics(False)
# root.enableDataDebug(False)

# root.enableAsic(AsicNum, True)

# ssdPath = "/u1/ePixHRM320k/acquisition"
# filename = f"{ssdPath}/LRtest1.dat"
# rate = 5000 # HZ
# frames = 10000

# root.acquireToFile(filename, frames, rate)

Acquisition started: filename: /u1/ePixHRM320k/acquisition/LRtest1.dat, rate: 5000, #frames:10000
Stop Triggers command executed
Set Auto Trigger command executed
Start Auto Trigger command executed
Triggers sent: 10000
Stop Triggers command executed
Received on channel 0 0 frames...
Received on channel 1 7706 frames...
Received on channel 2 0 frames...
Received on channel 3 0 frames...
Waiting for file to close...
Acquisition complete and file closed


In [ ]:
# stops the spamming of messages
numberOfTriggers = 1000
data = readoutFrames(dataDebug, numberOfTriggers, chargeInj=False)

In [ ]:
# Plot raw data and histogram
dataTimeAvg = np.average(data,axis=2)

print(np.shape(dataTimeAvg))

plt.figure(figsize = (10,4))
plt.imshow(dataTimeAvg, interpolation='none', aspect='auto', origin='lower', vmax=2**15)
plt.ylabel("Row [#]")
plt.xlabel("Col [#]")
plt.title("Raw data - Avg: {:}".format(getMedianDark(data)))
plt.colorbar()

plt.figure(figsize = (10,5))
plt.hist(dataTimeAvg.flatten(), range=[0,2**15],bins=2**8, log=True)
plt.title("Distribution of dark values")
plt.ylabel("Entries [#]")
plt.xlabel("ADC Value [ADUs]")

In [ ]:
# Uncomment to filter bad pixels!

filterBad = True

if filterBad:
    data_filtered, numBadPixels = filter_bad_pixels(data,3)
else:
    data_filtered, numBadPixels = data, 0


print(numBadPixels)

# Calculate data_noisemap
data_noisemap = np.std(data_filtered, axis=2)

# Flatten array
noisemap_flat = data_noisemap.flatten()

# Remove 0 noise (lanes not locked)
noisemap_flat = noisemap_flat[noisemap_flat > 0]

median_noise = np.median(noisemap_flat)
std_noise    = np.std(noisemap_flat)

# Get outliers (calculated as *2 noise)
noisemap_outliers = noisemap_flat[noisemap_flat > median_noise*2]

print('Noise: {:3f}'.format(median_noise))

# Top plot
plt.figure(figsize=(12,5))
plt.imshow(data_noisemap, interpolation='none', aspect='auto', vmin=0, vmax=median_noise*3, origin='lower')
plt.ylabel("Row [#]")
plt.xlabel("Col [#]")
plt.title(r'Noise: {:.1f} $\pm$ {:.1f} - Found and disabled {:.0f} hot pixels [{:.1f}% of whole matrix]'.format(median_noise, std_noise, numBadPixels, numBadPixels*100/(192.0*384.0)))
plt.colorbar()

# Bottom plot
fig, ax = plt.subplots(1,2, figsize=(13,3))

ax[0].hist(data_noisemap[(data_noisemap < 400) & (data_noisemap > 1)].flatten(),bins=2**8)
ax[0].set_title("Pixel noise distribution - Zoom on median")
ax[0].set_ylabel("Entries [#]")
ax[0].set_xlabel("Noise [ADUs]")

ax[1].hist(data_noisemap[(data_noisemap > 1)].flatten(),bins=2**8,log=True)
ax[1].set_title("Pixel noise distribution - Whole range")
ax[1].set_ylabel("Entries [#]")
ax[1].set_xlabel("Noise [ADUs]")



In [ ]:
# Subtract mean and display image
dataDarkSub = data_filtered - np.median(data_filtered, axis=2, keepdims=True)

# Trim range here if needed
dataDarkSub = dataDarkSub[(dataDarkSub > -1000)]

plt.figure(figsize = (10,5))
plt.hist(dataDarkSub.flatten(),bins=2**8)
plt.ylabel("Row [#]")
plt.xlabel("Col [#]")
plt.title("Dark-subtracted image")

# Run when we are done

In [ ]:
# Disable data debug so it does not store data infinitely and crash software
dataDebug.cleanData()
print(dataDebug.getData().shape)
root.enableDataDebug(False)